## Notebook for 1D BDT scan and calculating FOMs in Bd2psi2SKS on data


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/chasenberg/repos/')
import os
import sys
import ROOT
from ROOT import TFile, TH1D, TH2D, TCanvas, gStyle, TLine, TTree
from ROOT import (RooArgSet, RooRealVar, RooDataSet, RooPlot, RooFit, RooStats, RooArgList)
import root_pandas as rp
import root_numpy as ry 

import math
import itertools
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from root_pandas import read_root
import pandas as pd

sys.path.append('/home/vmueller/repos')
from dopy.doroot.root_utils import read_roodataset_from_tree, plot_pulls
from dopy.doplot.plotting import Plotter, Plot                

In [2]:

from ROOT import RooRealVar, RooExponential, RooExtendPdf, RooAddPdf, RooArgList, RooGaussian
from ROOT import RooFormulaVar
from ROOT import gSystem
gSystem.Load('libRooFit.so')
gSystem.Load('/home/chasenberg/repos/dopy/dolib/CustomShapes/libKll')
from ROOT import RooIpatia2

### Read ROOT file

In [3]:
dataset_file_name = '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/Bd2JpsiKS_selected.root'

In [4]:
branches = [
            'B0_TAGOMEGA_OS',
            'B0_TAGDECISION_OS',
            'B0_TAU',
            'B0_TAUERR',
            'B0_TRUETAU',
            'B0_FitPVConst_MinIPCHI2anyPV_flat',
            'idxRunNumber',
            'idxEventNumber',
            'idxPV'
            ]

In [5]:
l0_cut =  '((abs(1115.683-varLambda0MassHypo_ppluspiminus)>17)|piplus_ProbNNp<0.4)&((abs(1115.683-varLambda0MassHypo_pminuspiplus)>17)|(piminus_ProbNNp<0.4))'

In [6]:
tree_mc = 'Bd2JpsiKS'
signal_dataframe_wrongPV  = rp.read_root(dataset_file_name,key=tree_mc,columns=branches,where=l0_cut, flatten=False)
signal_dataframe_wrongPV  = signal_dataframe_wrongPV.replace([np.inf, -np.inf], np.nan)
signal_dataframe_wrongPV  = signal_dataframe_wrongPV.dropna()

In [7]:
signal_dataframe_wrongPV['Delta_TAU'] = signal_dataframe_wrongPV['B0_TAU'] -signal_dataframe_wrongPV['B0_TRUETAU']

In [8]:
signal_dataframe_wrongPV.to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root',key='Bd2JpsiKS')

In [9]:
'''test_1 = pd.DataFrame()
test_2 = pd.DataFrame()
test_3 = pd.DataFrame()
for row in signal_dataframe_wrongPV.itertuples():
    print("yes")
pv_num = signal_dataframe_wrongPV['runNumber'].unique()
pv_num = np.array(pv_num)
print(len(pv_num))
for nums in pv_num:
    num = str(nums)
    test_1 = signal_dataframe_wrongPV.query('runNumber'+'=='+num)
    evt_num = test_1['eventNumber'].unique()
    evt_num = np.array(evt_num)
    print(len(evt_num))
    for evts in evt_num:
        print(len(evt_num))
        evt = str(evts)
        #print(test_1.query('eventNumber'+'=='+evt).shape).sample(n=1)
        test_3.append(test_1[test_1['eventNumber']==evts]).sample(n=1) 
    for evts in evt_num:
        print(len(evt_num))
        evt = str(evts)
        #print(test_1.query('eventNumber'+'=='+evt).shape).sample(n=1)
        test_3.append(test_1[test_1['eventNumber']==evts]).sample(n=1)'''

'test_1 = pd.DataFrame()\ntest_2 = pd.DataFrame()\ntest_3 = pd.DataFrame()\nfor row in signal_dataframe_wrongPV.itertuples():\n    print("yes")\npv_num = signal_dataframe_wrongPV[\'runNumber\'].unique()\npv_num = np.array(pv_num)\nprint(len(pv_num))\nfor nums in pv_num:\n    num = str(nums)\n    test_1 = signal_dataframe_wrongPV.query(\'runNumber\'+\'==\'+num)\n    evt_num = test_1[\'eventNumber\'].unique()\n    evt_num = np.array(evt_num)\n    print(len(evt_num))\n    for evts in evt_num:\n        print(len(evt_num))\n        evt = str(evts)\n        #print(test_1.query(\'eventNumber\'+\'==\'+evt).shape).sample(n=1)\n        test_3.append(test_1[test_1[\'eventNumber\']==evts]).sample(n=1) \n    for evts in evt_num:\n        print(len(evt_num))\n        evt = str(evts)\n        #print(test_1.query(\'eventNumber\'+\'==\'+evt).shape).sample(n=1)\n        test_3.append(test_1[test_1[\'eventNumber\']==evts]).sample(n=1)'

In [10]:
#test.shape,signal_dataframe_wrongPV.query('idxPV==0').shape,signal_dataframe_wrongPV.shape

### Function to calculate the FOMs

### Configure  and define cuts 

In [11]:
perform_dilution_scan=False

In [44]:
lowerbound=5220 # rather arbitrary signal window definition
upperbound=5450
#B0_FitDaughtersConst_M.setRange('SIGREGION',lowerbound,upperbound)

subdir = '/home/chasenberg/plots/wrongpv_cuttuning'
plot_dir    = os.path.join(subdir, 'plots')
results_dir = os.path.join(subdir, 'results')
plot_file   = os.path.join(plot_dir, 'all_plots.pdf')


if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
mass_var_name = 'B0_FitDaughtersConst_M'


if perform_dilution_scan==True:
    cuts = np.linspace(0.00001,0.02,100)  
    cuttype = 'Delta_TAU'
else:    
    cuts = np.linspace(0,15000,40)  
    cuttype = 'B0_FitPVConst_MinIPCHI2anyPV_flat'

cuts_for_which_to_store_sdata = []   

num_sig = []
num_bkg = []
num_all = []

cut_to_performance = {} 
cut_to_shape = {}

perform_fit = True
do_splot    = True
postfix = ''
final_params_file = None
oldParameters = None
final_model = None

### Calculate FOM

In [45]:
def calculate_foms(data_fit_cut,best_pv): 
    import math
    
    signal_sum    = 0
    cbkg_sum      = 0
    signal_sumw2  = 0
    signal_sum_wrong    = 0
    cbkg_sum_wrong       = 0
    signal_sumw2_wrong   = 0
    
    delta_m = 5065 
    S = 0.691 #sin2beta value (average from HFAG)
    
    timeres_power_bestPV = 0
    timeres_power_randomPV = 0
    

    
    tagomega = np.array(best_pv['B0_TAGOMEGA_OS'])
    tag1 = np.array(best_pv['B0_TAGDECISION_OS'])
    time = np.array(best_pv['B0_TAU']) 
    error = np.absolute(np.array(best_pv['Delta_TAU']))
    
    tagomega_wrong = np.array(data_fit_cut['B0_TAGOMEGA_OS'])
    tag1_wrong = np.array(data_fit_cut['B0_TAGDECISION_OS'])
    time_wrong = np.array(data_fit_cut['B0_TAU']) 
    error_wrong = np.absolute(np.array(data_fit_cut['Delta_TAU']))
    
    range_best_pv = best_pv.shape[0]-1
    
    for i in range(range_best_pv):
        signal_weight =  1 
        cbkg_weight   =  1 
        mistag        =  tagomega[i] 
        tag           =  tag1[i]
        decaytime     =  time[i]
        timeerror     =  error[i]
        
        
        signal_sum    += signal_weight
        signal_sumw2  += signal_weight**2
        cbkg_sum      += cbkg_weight

        timeerror_dilution = math.exp(-(delta_m*timeerror)**2)

        timeres_power_bestPV    += timeerror_dilution*signal_weight
         
    print('+++++++++++++++++++++++++')
    print(cut)
    timeerror_dilution_theory = math.exp(-(delta_m*cut)**2)

    timeres_power_bestPV /= signal_sum
    #timeres_power_wrong /= signal_sum_wrong

    sig_yields_cut = data_fit_cut.shape[0] 
    efficiency = sig_yields_cut / sig_yields_init
    
    print("The efficiency of the cut is:")
    print(efficiency)
    
    fomname_to_value = {
        'TimeresPower_bestPV' : timeres_power_bestPV,
        'TimeresTheory': timeerror_dilution_theory,
        'Efficiency'   : efficiency
    }
    #return fomname_to_value

    for i in range(data_fit_cut.shape[0]):
        signal_weight_wrong =  1 
        cbkg_weight_wrong   =  1 
        mistag_wrong        =  tagomega_wrong[i] 
        tag_wrong           =  tag1_wrong[i]
        decaytime_wrong     =  time_wrong[i]
        timeerror_wrong     =  error_wrong[i]
        
        
        signal_sum_wrong    += signal_weight_wrong
        signal_sumw2_wrong  += signal_weight_wrong**2
        
        timeerror_dilution_wrong = math.exp(-(delta_m*timeerror_wrong)**2)
    
        timeres_power_randomPV     += timeerror_dilution_wrong *signal_weight_wrong 
        
    timeres_power_randomPV /= signal_sum_wrong  
    fomname_to_value.update({'TimeresPower_randomPV' : timeres_power_randomPV})
    
    return fomname_to_value

In [ ]:
eff = signal_dataframe_wrongPV.query('idxPV==0').query('B0_FitPVConst_MinIPCHI2anyPV_flat>15000').shape[0] / signal_dataframe_wrongPV.query('idxPV==0').shape[0]
print(eff)

0.634263413152473


In [ ]:
from ROOT import RooArgList
import subprocess, os
import time

x=None



if perform_dilution_scan==True:
    for cut in cuts:
        print('INFO: Starting to test cut ' + '{0}<{1}'.format(cuttype,cut), flush=True)
        data_fit_cut = signal_dataframe_wrongPV.query('{0}>-{1}&{0}<{1}'.format(cuttype,cut)) #data_after_import.reduce('{0}>{1}'.format(cuttype,cut))
        best_pv = signal_dataframe_wrongPV.query('{0}>-{1}&{0}<{1}'.format(cuttype,cut)).query('idxPV==0')
        #data_fit_cut = data_fit_cut.query('B0_FitPVConst_MinIPCHI2anyPV_flat>6')
        #print("The size of the sample is:")
        #print(data_fit_cut['Delta_TAU'].min())
        #data_fit_cut.query('B0_FitPVConst_MinIPCHI2anyPV_flat>15').to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root',key=tree_mc)
        data_fit_cut.to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root',key=tree_mc)
        print('INFO: Call CandidateSelectionGrimReaper', flush=True)
        my_env = os.environ.copy()
        my_env['PATH'] = '/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/home/chasenberg/bin'
        my_env['LD_LIBRARY_PATH'] = '/usr/local/lib'
        my_env['script'] = '/home/chasenberg/repos/b2cc_sin2beta_run2/notebooks/selection'
        my_command = 'source /lhcbsoft/LHCbSoftwareSetup.sh &&' \
        'lb-run DaVinci/v41r2 $BASH -c "' \
        'echo $PATH && ' \
        'source /doosoft/InstallDooSoftware/LoadDooSoftware && CandidateSelectionGrimReaper /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root Bd2JpsiKS /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root Bd2JpsiKS "idxRunNumber""'        
        subprocess.Popen([my_command], env=my_env, shell=True)
   # subprocess.call(('bash /home/chasenberg/repos/b2cc_sin2beta_run2/notebooks/selection/random_sel.sh'),shell=True)                      
        print("Wait for GrimReaper")
        time.sleep(40.0)        # pause 5.5 seconds
        print("GrimReaper hopefully finished")
        print('INFO: Read selected data', flush=True)
        data_fit_cut = rp.read_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root',key=tree_mc)
        print('INFO: Start Calculation', flush=True)
        #print('INFO: Building Ipatia (signal) + Exp (combinatorics) + Ipatia (bs-component) model for data', flush=True) 
        #final_model = build_mass_model(B0_FitDaughtersConst_M)
    
        cut_to_shape[cut] = {} 
        cut_to_performance[cut] = {}
   
    
        foms = calculate_foms(data_fit_cut,best_pv)
        cut_to_performance[cut].update(foms)
    
        print('INFO: Finish Calculation', flush=True)
else:
    data_fit_cut = signal_dataframe_wrongPV 
    my_env = os.environ.copy()
    my_env['PATH'] = '/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/home/chasenberg/bin'
    my_env['LD_LIBRARY_PATH'] = '/usr/local/lib'
    my_env['script'] = '/home/chasenberg/repos/b2cc_sin2beta_run2/notebooks/selection'
    my_command = 'source /lhcbsoft/LHCbSoftwareSetup.sh &&' \
        'lb-run DaVinci/v41r2 $BASH -c "' \
        'echo $PATH && ' \
        'source /doosoft/InstallDooSoftware/LoadDooSoftware && CandidateSelectionGrimReaper /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root Bd2JpsiKS /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root Bd2JpsiKS "idxRunNumber""'        
    subprocess.Popen([my_command], env=my_env, shell=True)
        
    print("Wait for GrimReaper")
    time.sleep(40.0)        # pause 5.5 seconds
    print("GrimReaper hopefully finished")
    print('INFO: Read selected data', flush=True)
    data_fit_cut = rp.read_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root',key=tree_mc)
    sig_yields_init = data_fit_cut.shape[0]
    
    for cut in cuts:
        print('INFO: Starting to test cut ' + '{0}>{1}'.format(cuttype,cut), flush=True)
        data_fit_cut = signal_dataframe_wrongPV.query('{0}>{1}'.format(cuttype,cut)) 
        best_pv = signal_dataframe_wrongPV.query('{0}>{1}'.format(cuttype,cut)).query('idxPV==0')
        data_fit_cut.to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root',key=tree_mc)
        print('INFO: Call CandidateSelectionGrimReaper', flush=True)
        
        my_env = os.environ.copy()
        my_env['PATH'] = '/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/home/chasenberg/bin'
        my_env['LD_LIBRARY_PATH'] = '/usr/local/lib'
        my_env['script'] = '/home/chasenberg/repos/b2cc_sin2beta_run2/notebooks/selection'
        my_command = 'source /lhcbsoft/LHCbSoftwareSetup.sh &&' \
        'lb-run DaVinci/v41r2 $BASH -c "' \
        'echo $PATH && ' \
        'source /doosoft/InstallDooSoftware/LoadDooSoftware && CandidateSelectionGrimReaper /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root Bd2JpsiKS /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root Bd2JpsiKS "idxRunNumber""'        
        subprocess.Popen([my_command], env=my_env, shell=True)
        
        print("Wait for GrimReaper")
        time.sleep(40.0)        # pause 5.5 seconds
        print("GrimReaper hopefully finished")
        print('INFO: Read selected data', flush=True)
        data_fit_cut = rp.read_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_random.root',key=tree_mc)
        print('INFO: Start Calculation', flush=True)
        sig_yields_cut = data_fit_cut.shape[0] 
        efficiency = sig_yields_cut / sig_yields_init
        print("The inital number of candidates is:")
        print(sig_yields_init)
        print("----------------------------------")
        print("The number of candidates after the cut is:")
        print(sig_yields_cut)
        print("----------------------------------")
        print("The efficiency of the cut is:")
        print(efficiency)
        cut_to_shape[cut] = {} 
        cut_to_performance[cut] = {}
   
    
        foms = calculate_foms(data_fit_cut,best_pv)
        cut_to_performance[cut].update(foms)
    
        print('INFO: Finish Calculation', flush=True)

Wait for GrimReaper
GrimReaper hopefully finished
INFO: Read selected data
INFO: Starting to test cut B0_FitPVConst_MinIPCHI2anyPV_flat>0.0
INFO: Call CandidateSelectionGrimReaper
Wait for GrimReaper
GrimReaper hopefully finished
INFO: Read selected data
INFO: Start Calculation
The inital number of candidates is:
459693
----------------------------------
The number of candidates after the cut is:
501573
----------------------------------
The efficiency of the cut is:
1.0911042804654412
+++++++++++++++++++++++++
0.0
The efficiency of the cut is:
1.0911042804654412
INFO: Finish Calculation
INFO: Starting to test cut B0_FitPVConst_MinIPCHI2anyPV_flat>384.61538461538464
INFO: Call CandidateSelectionGrimReaper
Wait for GrimReaper
GrimReaper hopefully finished
INFO: Read selected data
INFO: Start Calculation
The inital number of candidates is:
459693
----------------------------------
The number of candidates after the cut is:
454640
----------------------------------
The efficiency of the c

In [ ]:
import numpy as np

def unpack_cut_dict(cut_dict, normalize=True):
    cuts = np.array(list(cut_dict.keys()))
    cuts.sort()
    
    new_dict = {}
    for cut in cuts:
        parameters_dict = cut_dict[cut]
        for parameter, value in parameters_dict.items():
            if parameter in new_dict:
                new_dict[parameter].append(value)
            else:
                new_dict[parameter] = [value]
    
    for parameter, values in new_dict.items():
        new_dict[parameter] = np.array(values)
        if normalize:
            new_dict[parameter] = values/np.max(np.abs(values))
    
    return cuts, new_dict

x, ys = unpack_cut_dict(cut_to_performance)
for parameter, values in ys.items():
    if "value" in parameter:
        print(values)
        y_errlo = np.array(ys[parameter.replace("_value","_errlo")])
        y_errhi = np.array(ys[parameter.replace("_value","_errhi")])
        y_errors= abs(np.vstack([y_errlo,y_errhi]))
        plt.errorbar(x,values,yerr=y_errors,marker='.',linestyle='', label='sigma')
        plt.xlabel(parameter)
        plotname = parameter + '.pdf'
        singleplot_file = os.path.join(plot_dir, plotname)
        plt.savefig(singleplot_file)
        plt.show()

### Plot FOMs

In [ ]:
if perform_dilution_scan==True:
    import random
    import seaborn as sns
    sns.set_style("whitegrid")
    sns.set_style("whitegrid", {"legend.frameon": True})

    x, ys = unpack_cut_dict(cut_to_performance)
    shift = 0.0

    foms_to_plot = {}
    #foms_to_plot["TaggingPower"] = r"Flavour Tagging"
    foms_to_plot["TimeresPower_bestPV"] = "Best PV selection"
    foms_to_plot["TimeresPower_randomPV"]   = "Random PV selection"
    #foms_to_plot["BsFOM"]        = "$X_i$-term"
    #foms_to_plot["BdFOM_p"]      = "FOM ${}_{B^0_d}$"
    #foms_to_plot["BdFOM"]        = "FOM"

    #for parameter, values in ys.items():
    for parameter in foms_to_plot:
        values=ys[parameter]
        #if "FOM" in parameter or "EffSig" in parameter or "s3_over_spb2" in parameter\
        #or "TaggingPower" in parameter or "TimeresPower" in parameter:
        if parameter in foms_to_plot:
            plt.errorbar(x,values+random.random()*shift,marker='.',linestyle='-',label=foms_to_plot[parameter])
            plt.xlabel("Delta_TAU cut")

    plt.gca().set_ylim(0.94,1.001)
    plt.gca().set_xlim(0.0000000001,0.02)
    plt.legend(loc='best')
    plt.savefig(subdir + '/dilution_randomsel_no_minipchi2_cut.pdf')
    plt.show()

In [ ]:
print(ys['TimeresPower_bestPV'])

In [ ]:
print(ys['TimeresPower_randomPV'])

In [ ]:
print(x)

Plotting of B0_TAU and Delta_TAU

In [ ]:
if perform_dilution_scan==False:
    import seaborn as sns; sns.set_style('white')

    x, ys = unpack_cut_dict(cut_to_performance)
    test = pd.DataFrame()
    test['TimeresPower_randomPV'] = np.array(ys['TimeresPower_randomPV'])
    test['Efficiency'] = np.array(ys['Efficiency'])
    y_lim = (0.9,1)
    x_lim = (0.996,1.001)
    g = sns.jointplot(x='TimeresPower_randomPV', y='Efficiency',data=test,stat_func=None,space=0.05, kind="kde",xlim=x_lim,ylim=y_lim, color="r")
    #g.savefig("/home/chasenberg/plots/wrong_pv/deltatau_vs_pvzpull.pdf")
    print(test.shape)

In [ ]:
x = test['TimeresPower_randomPV'] = np.array(ys['TimeresPower_randomPV'])
y = test['Efficiency'] = np.array(ys['Efficiency'])
lm = sns.lmplot(x='TimeresPower_randomPV', y='Efficiency',data=test,x_estimator=np.mean)
axes = lm.axes
axes[0,0].set_ylim(0.4,1)
axes[0,0].set_xlim(0.99,1.05)

In [ ]:
 np.array(ys['TimeresPower_randomPV']).min()

In [ ]:
a = test['Efficiency']
plt.plot(cuts,a)


In [ ]:
a = test['TimeresPower_randomPV']
plt.plot(cuts,a)